# Integrate Cleanlab with AWS Strands Agents

This tutorial demonstrates the easiest way to validate and improve the trustworthiness of AI Agents built with the [Strands SDK](https://github.com/strands-agents/sdk-python). With *minimal* changes to your existing Strands Agent code, you can detect bad responses and automatically remediate them in real-time.

## Setup

The Python packages required for this tutorial can be installed using pip:

In [ ]:
%pip install --upgrade cleanlab-codex strands-agents "strands-agents[openai]" tavily-python

This tutorial requires a Cleanlab API key. Get one [here](https://codex.cleanlab.ai/).

In [ ]:
import os
os.environ["CODEX_API_KEY"] = "<Cleanlab Codex API key>"  # Get your API key from: https://codex.cleanlab.ai/
os.environ["OPENAI_API_KEY"] = "<OpenAI API key>"  # for using OpenAI models with Strands
os.environ["TAVILY_API_KEY"] = "<TAVILY API KEY>"  # for using a web search tool (get your free API key from Tavily)

In [7]:
from cleanlab_codex.client import Client
from tavily import TavilyClient

## Overview of this tutorial

This tutorial showcases using Cleanlab's CleanlabModel wrapper to add real-time validation to Strands Agents. 

We'll demonstrate four key scenarios:

1. **Conversational Chat Response** - Basic agent interaction with validation
2. **Tool Call Response** - Agent response using tools with validation
3. **Bad AI Response** - How Cleanlab prevents problematic responses and cleans message history
4. **Information Retrieval Tool Call Response** - Context-aware validation with web search

### Example Use Case: Bank Loan Customer Support

We'll build a customer support agent for bank loans to demonstrate validation scenarios.

Let's define tools representing different response quality levels:
- a *good* tool that returns reasonable information
- a *bad* tool that returns problematic information
- a web search tool that provides additional *context* to the Agent

**Note:** The web search tool follows the example information retrieval function defined in the [Strands Web Search tutorial](https://aws.amazon.com/blogs/machine-learning/build-dynamic-web-research-agents-with-the-strands-agents-sdk-and-tavily/)

In [8]:
# Tool definitions for demonstration scenarios
from strands.tools.decorator import tool

# ============ Good Tool: Returns reasonable information ============
@tool
def get_payment_schedule(account_id: str) -> str:
    """Get payment schedule for an account."""
    payment_schedule = f"""Account {account_id} has: 
    - Bi-weekly payment plan
    - Upcoming payment scheduled for next Friday
    """
    return payment_schedule

# ============ Bad Tool: Returns problematic information ============
@tool
def get_total_amount_owed(account_id: str) -> dict:
    """A tool that simulates fetching the total amount owed for a loan.
    **Note:** This tool returns a hardcoded *unrealistic* total amount for demonstration purposes."""
    return {
        "account_id": account_id,
        "currency": "USD",
        "total": 7000000000000000000000000000000000000.00,
    }

# ============ Web Search Tool: Provides context for the Agent ============
@tool
def web_search(
    query: str, time_range: str | None = None, include_domains: str | None = None
) -> str:
    """Perform a web search. Returns the search results as a string, with the title, url, and content of each result ranked by relevance.
    Args:
        query (str): The search query to be sent for the web search.
        time_range (str | None, optional): Limits results to content published within a specific timeframe.
            Valid values: 'd' (day - 24h), 'w' (week - 7d), 'm' (month - 30d), 'y' (year - 365d).
            Defaults to None.
        include_domains (list[str] | None, optional): A list of domains to restrict search results to.
            Only results from these domains will be returned. Defaults to None.
    Returns:
        formatted_results (str): The web search results
    """
    
    def format_search_results_for_agent(search_results: list[dict]) -> str:
        """Format search results into a numbered context string for the agent."""
        results = search_results["results"]
        parts = []
        for i, r in enumerate(results, start=1):
            title = r.get("title", "").strip()
            content = r.get("content", "").strip()
            if title or content:
                block = (
                    f"Context {i}:\n"
                    f"title: {title}\n"
                    f"content: {content}"
                )
                parts.append(block)
        return "\n\n".join(parts)
    
    client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    formatted_results = format_search_results_for_agent(
        client.search(
            query=query,
            max_results=2,
            time_range=time_range,
            include_domains=include_domains
        )
    )
    return formatted_results

## Create Cleanlab Project

To use the Cleanlab AI Platform for validation, we must first [create a Project](/codex/web_tutorials/create_project/).
Here we assume no (question, answer) pairs have already been added to the Project yet.

User queries where Cleanlab detected a bad response from your AI app will be logged in this Project for SMEs to later answer.

In [9]:
# Create a Cleanlab project
client = Client()

project = client.create_project(
    name="Strands Agent with Cleanlab Validation Tutorial",
    description="Tutorial demonstrating validation of a Strands Agent with CleanlabModel wrapper"
)

## Strands Integration

To add validation to your Strands agents, wrap any existing Strands model with a CleanlabModel for real-time validation during Agent execution.

Cleanlab's wrapper intercepts responses during generation and validates them in real-time, and provides automatic expert answer substitution and guardrail enforcement.

**Integration steps:**
1. Wrap your Model with CleanlabModel 
2. Create your Agent with the wrapped Model
3. Call `cleanlab_model.set_agent_reference(agent)` for full functionality

#### Context-Aware Validation for Information Retrieval

For agents with tools that retrieve information (e.g., RAG, web search, database queries), Cleanlab can use this retrieved content as **context** during validation. This enables more accurate evaluation by:

- Checking if the AI response is grounded in the retrieved information
- Measuring context sufficiency (whether enough information was retrieved)
- Detecting hallucinations by comparing the response against actual context

To enable this, specify the names of your context-providing tools in the `context_retrieval_tools` parameter during CleanlabModel initialization.

In [10]:
import uuid

from strands.agent.agent import Agent
from strands.models.openai import OpenAIModel
from strands.session.file_session_manager import FileSessionManager

from cleanlab_codex.experimental.strands import CleanlabModel

SYSTEM_PROMPT = "You are a customer service agent. Be polite and concise in your responses."
FALLBACK_RESPONSE = "Sorry I am unsure. You can try rephrasing your request."

# Create base model
base_model = OpenAIModel(
    model_id="gpt-4o-mini",
)

### New code to add for Cleanlab API ###
cleanlab_model = CleanlabModel( # Wrap with Cleanlab validation
    underlying_model=base_model,
    cleanlab_project=project,
    fallback_response=FALLBACK_RESPONSE,
    context_retrieval_tools=["web_search", "get_payment_schedule", "get_total_amount_owed"]  # Specify tool(s) that provide context
)
### End of new code to add for Cleanlab API ###

# Create agent with validated model for normal conversation
agent = Agent(
    model=cleanlab_model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_payment_schedule, get_total_amount_owed, web_search],
    session_manager=FileSessionManager(session_id=uuid.uuid4().hex),  # Persist chat history
)

### New code to add for Cleanlab API ###
cleanlab_model.set_agent_reference(agent)
### End of new code to add for Cleanlab API ###

### Scenario 1: Conversational Chat Response

Let's start with a basic agent interaction without tools. 

The CleanlabModel wrapper validates the response in real-time.

**Optional: Helper method to prompt the agent and print validation results**



In [11]:

def run(agent: Agent, query: str):
    print(f"Query: '{query}'")
    print("Response: ", end="")

    # Prompt the agent and get response
    result = agent(query)
    print()

    # Show tool usage metrics
    if hasattr(result, 'metrics') and hasattr(result.metrics, 'tool_metrics'):
        if len(result.metrics.tool_metrics) > 0:
            print(f"\n--- Historical Tool Usage ---")
            for tool_name, metrics in result.metrics.tool_metrics.items():
                print(f"Tool '{tool_name}': called {metrics.call_count} time(s), {metrics.success_count} successful")

    # Access validation results
    validation_results = agent.state.get('cleanlab_validation_results')
    print(f"\n--- Cleanlab Validation Results ---")
    print(f"Should Guardrail: {validation_results.get('should_guardrail', 'N/A')}")
    print(f"Escalated to SME: {validation_results.get('escalated_to_sme', 'N/A')}")
    print(f"Expert Answer Available: {bool(validation_results.get('expert_answer'))}")
    print(f"Is Bad Response: {validation_results.get('is_bad_response', 'N/A')}")
            
    # Show eval scores if available
    if 'eval_scores' in validation_results:
        print(f"\n--- Key Evaluation Scores ---")
        eval_scores = validation_results['eval_scores']
        if 'trustworthiness' in eval_scores:
            trust_score = eval_scores['trustworthiness'].get('score', 'N/A')
            print(f"Trustworthiness: {trust_score}")
        if 'response_helpfulness' in eval_scores:
            help_score = eval_scores['response_helpfulness'].get('score', 'N/A')
            print(f"Response Helpfulness: {help_score}")
    

In [12]:
print("=== Scenario 1: General Knowledge Prompt Response ===")
run(agent, "What is a credit score?")

=== Scenario 1: General Knowledge Prompt Response ===
Query: 'What is a credit score?'
Response: A credit score is a numerical representation of an individual's creditworthiness, which reflects their ability to repay borrowed money. It typically ranges from 300 to 850, with higher scores indicating better creditworthiness. Credit scores are calculated based on credit history, including payment history, the amount of debt owed, length of credit history, types of credit used, and new credit inquiries. Lenders use credit scores to assess the risk of lending money or extending credit to individuals.

--- Cleanlab Validation Results ---
Should Guardrail: False
Escalated to SME: False
Expert Answer Available: False
Is Bad Response: False

--- Key Evaluation Scores ---
Trustworthiness: 0.9847615785344598
Response Helpfulness: 0.9975124378110127


### Scenario 2: Tool Call Response

Now let's test an agent interaction that uses tools. Cleanlab validation checks both tool usage and the final response.

In [14]:
print("=== Scenario 2: Tool Call Prompt (Successful) ===")
run(agent, "What is the payment schedule for account ID 12345?")

=== Scenario 2: Tool Call Prompt (Successful) ===
Query: 'What is the payment schedule for account ID 12345?'
Response: 
Tool #1: get_payment_schedule
The payment schedule for account ID 12345 is as follows:

- **Payment Plan:** Bi-weekly
- **Next Payment:** Scheduled for next Friday.

--- Historical Tool Usage ---
Tool 'get_payment_schedule': called 1 time(s), 1 successful

--- Cleanlab Validation Results ---
Should Guardrail: False
Escalated to SME: False
Expert Answer Available: False
Is Bad Response: False

--- Key Evaluation Scores ---
Trustworthiness: 0.9997877497484022
Response Helpfulness: 0.9974053294473002


After this interaction, we can see the tool calls and response show up in the message history.

In [15]:
agent.messages

[{'role': 'user', 'content': [{'text': 'What is a credit score?'}]},
 {'role': 'assistant',
  'content': [{'text': "A credit score is a numerical representation of an individual's creditworthiness, which reflects their ability to repay borrowed money. It typically ranges from 300 to 850, with higher scores indicating better creditworthiness. Credit scores are calculated based on credit history, including payment history, the amount of debt owed, length of credit history, types of credit used, and new credit inquiries. Lenders use credit scores to assess the risk of lending money or extending credit to individuals."}]},
 {'role': 'user',
  'content': [{'text': 'What is the payment schedule for account ID 12345?'}]},
 {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'call_DVj7AXteouBjk9SNnyjnziDS',
     'name': 'get_payment_schedule',
     'input': {'account_id': '12345'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'call_DVj7AXteouBjk9SNnyjnziDS',
     

### Scenario 3: Bad AI Response

When an Agent calls an incorrect tool or summarizes problematic information returned from the tool call, Cleanlab automatically:

1. **Detects** the problematic response 
2. **Blocks** it from reaching the user
3. **Substitutes** a safe fallback response
4. **Cleans** message history to remove problematic tool calls

Let's see this in action:

In [16]:
print("=== Scenario 3: Bad AI Response ===")
run(agent, "How much do I owe on my loan for account ID 12345?")

=== Scenario 3: Bad AI Response ===
Query: 'How much do I owe on my loan for account ID 12345?'
Response: 
Tool #2: get_total_amount_owed
Sorry I am unsure. You can try rephrasing your request.

--- Historical Tool Usage ---
Tool 'get_payment_schedule': called 1 time(s), 1 successful
Tool 'get_total_amount_owed': called 1 time(s), 1 successful

--- Cleanlab Validation Results ---
Should Guardrail: True
Escalated to SME: True
Expert Answer Available: False
Is Bad Response: True

--- Key Evaluation Scores ---
Trustworthiness: 0.11115133435531592
Response Helpfulness: 0.9516016982255979


After this chat turn, we see the message history is updated only with the *user query* and *final agent response*.

In [17]:
agent.messages

[{'role': 'user', 'content': [{'text': 'What is a credit score?'}]},
 {'role': 'assistant',
  'content': [{'text': "A credit score is a numerical representation of an individual's creditworthiness, which reflects their ability to repay borrowed money. It typically ranges from 300 to 850, with higher scores indicating better creditworthiness. Credit scores are calculated based on credit history, including payment history, the amount of debt owed, length of credit history, types of credit used, and new credit inquiries. Lenders use credit scores to assess the risk of lending money or extending credit to individuals."}]},
 {'role': 'user',
  'content': [{'text': 'What is the payment schedule for account ID 12345?'}]},
 {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'call_DVj7AXteouBjk9SNnyjnziDS',
     'name': 'get_payment_schedule',
     'input': {'account_id': '12345'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'call_DVj7AXteouBjk9SNnyjnziDS',
     

### Scenario 4: Information Retrieval Tool Call Response

Now let's ask a question that requires our Agent to use web search, which we specified in our `context_retrieval_tools` list.

**What happens with context-aware validation:**
- Tool results are automatically passed to Cleanlab as context
- Cleanlab can evaluate whether the AI response is grounded in the retrieved information, represented with the Context Sufficiency score
- You'll see a "Retrieved Context" section in the Cleanlab Project UI showing what information was available for validation

In [18]:
print("=== Scenario 4: Information Retrieval Tool Call Response ===")
run(agent, "What is an upcoming event in San Francisco?")

=== Scenario 4: Information Retrieval Tool Call Response ===
Query: 'What is an upcoming event in San Francisco?'
Response: 
Tool #3: web_search
Here are some upcoming events in San Francisco:

1. **San Francisco Post Member Appreciation Event** - 2025
2. **San Francisco Post Annual Holiday Gala** - 2025
3. **Shucked** - September 9 – October 5, 2025, at Curran Theatre
4. **Gabby's Dollhouse Live!** - September 21, 2025

For more details, you may want to check the official sites or ticketing platforms.

--- Historical Tool Usage ---
Tool 'get_payment_schedule': called 1 time(s), 1 successful
Tool 'get_total_amount_owed': called 1 time(s), 1 successful
Tool 'web_search': called 1 time(s), 1 successful

--- Cleanlab Validation Results ---
Should Guardrail: False
Escalated to SME: False
Expert Answer Available: False
Is Bad Response: False

--- Key Evaluation Scores ---
Trustworthiness: 0.9365397071921228
Response Helpfulness: 0.9975071030420052


Context is now automatically extracted from web search tool result and passed to Cleanlab validation, improving evaluation accuracy for information retrieval scenarios.

In [19]:
agent.messages[-3:] # Last 3 messages to see web search tool call and context

[{'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'call_PqeMYxP5Lltzl5TeSvtfrX7O',
     'name': 'web_search',
     'input': {'query': 'upcoming events in San Francisco',
      'time_range': 'w'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'call_PqeMYxP5Lltzl5TeSvtfrX7O',
     'status': 'success',
     'content': [{'text': "Context 1:\ntitle: San Francisco Post Upcoming Events - SAME\ncontent: San Francisco Post Upcoming Events · San Francisco Post Member Appreciation Event (2025) · San Francisco Post Annual Holiday Gala (2025). Please join the San\n\nContext 2:\ntitle: BroadwaySF | Official Ticketing Site of Golden Gate, Orpheum, and ...\ncontent: Upcoming Events ; Shucked · September 9–October 5, 2025. Curran Theatre. Buy tickets for Shucked ; Gabby's Dollhouse Live! Presented by Walmart · September 21, 2025."}]}}]},
 {'role': 'assistant',
  'content': [{'text': "Here are some upcoming events in San Francisco:\n\n1. **San Francisco Post Member Apprec

## How Cleanlab Validation Works

Cleanlab evaluates AI responses across multiple dimensions (trustworthiness, helpfulness, reasoning quality, etc.) and provides scores, guardrail decisions, and expert remediation.

For detailed information on Cleanlab's validation methodology, see:
- [Cleanlab Validation Overview](/codex/tutorials/other_rag_frameworks/validator/)
- [Understanding Evaluation Metrics](/codex/tutorials/other_rag_frameworks/validator_conversational/#evaluation-metrics)
- [Configuring Validation Thresholds](/codex/web_tutorials/create_project/)

### Message History Management

When Cleanlab detects a problematic response that involved tool calls, it performs the following cleanup:

1. **Identifies the problematic turn**: Finds the conversation turn that produced the bad response
2. **Removes tool calls**: Eliminates the assistant message containing tool calls from history
3. **Removes tool results**: Eliminates the corresponding tool result messages from history
4. **Preserves user messages**: Keeps user queries to maintain conversation context
5. **Adds clean response**: Adds the safe fallback or expert answer to history

This prevents the problematic tool information from contaminating future conversation turns.

### Specifying Context Handling in More Detail

If you want more control over how context is passed to Cleanlab, it's recommended to create a custom CleanlabModel subclass and override the `cleanlab_get_validate_fields` method with custom logic to extract context from tool results and include in validation.

```python
from typing import Any
from strands.types.content import Messages
from cleanlab_codex.experimental.strands import CleanlabModel
from cleanlab_codex.experimental.strands.cleanlab_model import get_latest_user_message_content

def custom_get_context_function(messages: Messages) -> str:
    # Define your custom context extraction logic here
    return "your context extraction logic"

class CleanlabModelWithContext(CleanlabModel):
    def __init__(self, **init_args) -> None:
        super().__init__(**init_args)
    
    def cleanlab_get_validate_fields(self, messages: Messages) -> dict[str, Any]:
        """Extract fields from messages for cleanlab validation (overridden to also return context)."""
        user_message_content = get_latest_user_message_content(messages)
        context = custom_get_context_function(messages)  # User defined function to extract context
        return {
            "query": user_message_content,
            "context": context,
        }
```

## What's different if I'm using Amazon Bedrock models with Strands?

The CleanlabModel wrapper works with any Strands-compatible model provider. To use Amazon Bedrock:

```python
from strands.models.bedrock import BedrockModel

# Create Bedrock model
base_model = BedrockModel(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    params={"temperature": 0.1}
)

# Wrap with CleanlabModel
cleanlab_model = CleanlabModel(
    underlying_model=base_model,
    cleanlab_project=project,
)

# Use exactly as shown in the examples above
agent = Agent(model=cleanlab_model, tools=[...])
cleanlab_model.set_agent_reference(agent)
```

The validation behavior and message history management work identically across all model providers.

## Summary

This tutorial demonstrated integrating Cleanlab validation with AWS Strands Agents using the CleanlabModel wrapper. 

Key benefits:

- **Real-time validation** during response generation
- **Automatic remediation** with expert answers and fallbacks
- **Message history cleanup** to prevent contamination
- **Context-aware validation** for retrieval-based agents
- **Multi-model support** (OpenAI, Anthropic, Amazon Bedrock, etc.)

The CleanlabModel wrapper provides enterprise-grade safety with minimal code changes.